In [1]:
#Gerekli kütüphaneleri ekleme
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import stats

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as score

import seaborn as sn
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("yz_dataset.csv")

In [3]:
df

,Zaman damgası,Yaşınız ?,Cinsiyetiniz ?,Aşağıdaki renklerden en çok hangisini seviyorsunuz ?,Tütün veya Alkol kullanıyor musunuz ?,Aşağıdaki hayvanlardan en çok hangisini seviyorsunuz ?,Aşağıdaki müzik türlerinden en çok hangisini seviyorsunuz,Burcunuz ?,En sevdiğiniz mevsim hangisidir ?,Kendinizi nasıl tanımlarsınız?,Sevgiliniz veya eşiniz var mı ?,Hayal kurmayı sever misiniz ?,Aşağıdaki sporlardan en çok hangisini seviyorsunuz ?
0,08.05.2022 23:11:32,23,Erkek,Beyaz,Hayır,Kuş,İlahi - Ezgi,Balık,İlkbahar,İçedönük,Hayır,Evet,"Top ile oynanan (Futbol, Basketbol, Tenis vb.)"
1,08.05.2022 23:11:51,23,Erkek,Siyah,Evet,Kedi,Rock,İkizler,İlkbahar,Dışadönük,Evet,Evet,Yüzmek
2,08.05.2022 23:17:50,23,Erkek,Gri,Evet,Kedi,Klasik Müzik,Yengeç,Kış,İçedönük,Evet,Evet,"Bisiklet, Kaykay, Paten vb. sürmek"
3,08.05.2022 23:18:51,21,Erkek,Yeşil,Evet,Köpek,Rock,Aslan,İlkbahar,İçedönük,Hayır,Evet,"Top ile oynanan (Futbol, Basketbol, Tenis vb.)"
4,08.05.2022 23:19:23,22,Erkek,Mavi,Evet,Köpek,Rock,Başak,İlkbahar,İçedönük,Hayır,Evet,Yüzmek
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,12.05.2022 21:59:39,24,Kadın,Sarı,Evet,Balık,Pop,Başak,Yaz,Dışadönük,Evet,Hayır,"Top ile oynanan (Futbol, Basketbol, Tenis vb.)"
252,13.05.2022 09:46:40,22,Erkek,Mavi,Evet,Penguen,Rock,Oğlak,Yaz,İçedönük,Hayır,Evet,"Bisiklet, Kaykay, Paten vb. sürmek"
253,13.05.2022 17:30:58,27,Erkek,Turuncu,Evet,Kedi,Rock,Koç,Yaz,Dışadönük,Evet,Evet,"Top ile oynanan (Futbol, Basketbol, Tenis vb.)"
254,14.05.2022 10:25:42,18,Kadın,Mavi,Evet,Köpek,Rock,Terazi,Yaz,Dışadönük,Hayır,Evet,"Top ile oynanan (Futbol, Basketbol, Tenis vb.)"


In [4]:
#Timestamp çıkarıldı
df = df.drop("Zaman damgası", axis=1)

In [5]:
#Null veriler droplandı
df = df.dropna()

In [6]:
#Dataframe hakkında bilgi
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255 entries, 0 to 255
Data columns (total 12 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   Yaşınız ?                                                  255 non-null    int64 
 1   Cinsiyetiniz ?                                             255 non-null    object
 2   Aşağıdaki renklerden en çok hangisini seviyorsunuz ?       255 non-null    object
 3   Tütün veya Alkol kullanıyor musunuz ?                      255 non-null    object
 4   Aşağıdaki hayvanlardan en çok hangisini seviyorsunuz ?     255 non-null    object
 5   Aşağıdaki müzik türlerinden en çok hangisini seviyorsunuz  255 non-null    object
 6   Burcunuz ?                                                 255 non-null    object
 7   En sevdiğiniz mevsim hangisidir ?                          255 non-null    object
 8   Kendinizi nasıl tanı

In [7]:
#Sütunlar alındı
from sklearn.preprocessing import OneHotEncoder
columns = list(df.columns)
columns = columns[2:]
columns

['Aşağıdaki renklerden en çok hangisini seviyorsunuz ?',
 'Tütün veya Alkol kullanıyor musunuz ?',
 'Aşağıdaki hayvanlardan en çok hangisini seviyorsunuz ?',
 'Aşağıdaki müzik türlerinden en çok hangisini seviyorsunuz',
 'Burcunuz ?',
 'En sevdiğiniz mevsim hangisidir ?',
 'Kendinizi nasıl tanımlarsınız?',
 'Sevgiliniz veya eşiniz var mı ?',
 'Hayal kurmayı sever misiniz ?',
 'Aşağıdaki sporlardan en çok hangisini seviyorsunuz ?']

In [8]:
#Sütunlara onehotencoding uygulanarak categorical input işlenebilir duruma geldi.
df_ready = pd.DataFrame()
for col in columns:
    encoder = OneHotEncoder()
    onehotarray = encoder.fit_transform(df[[col]]).toarray()
    items = [f'{col}_{item}' for item in encoder.categories_[0]]
    df_ready[items] = onehotarray

In [9]:
df_ready_norm = df_ready.copy()

In [10]:
#Cinsiyet için label işlemi. Yaş için ordinal encoding.
conditions = [
    (df['Cinsiyetiniz ?'] == 'Erkek'),
    (df['Cinsiyetiniz ?'] == 'Kadın')
    ]

values = [0, 1]

conditions_2 = [
    (df['Yaşınız ?'] <= 20),
    (df['Yaşınız ?'] > 20) & (df['Yaşınız ?'] <= 30),
    (df['Yaşınız ?'] > 30)
    ]

values_2 = [0, 1, 2]
df['gender'] = np.select(conditions, values)
df['age'] = np.select(conditions_2, values_2)
df_ready['age'] = np.array(df['age'], dtype=float)
df_ready['gender'] = np.array(df['gender'], dtype=float)

In [11]:
#Normalize değerler ile test için
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler().fit(df[['Yaşınız ?']])
df_ready_norm['age']  = norm.transform(df[['Yaşınız ?']])
df_ready_norm['gender'] = np.array(df['gender'], dtype=float)

In [12]:
#Dataframe shuffle landı.
df_ready = df_ready.sample(frac=1).reset_index(drop=True)
df_ready_norm = df_ready_norm.sample(frac=1).reset_index(drop=True)

In [13]:
#Train test Split
msk = np.random.rand(len(df_ready)) < 0.8
train = df_ready[msk].reset_index(drop=True)
test = df_ready[~msk].reset_index(drop=True)

train_norm = df_ready_norm[msk].reset_index(drop=True)
test_norm = df_ready_norm[~msk].reset_index(drop=True)

In [14]:
train.shape, test.shape

((210, 58), (45, 58))

In [15]:
x_train, y_train = train[train.columns[:-1]], train[train.columns[-1]]
x_test, y_test = test[test.columns[:-1]], test[test.columns[-1]]

x_train_norm, y_train_norm = train_norm[train_norm.columns[:-1]], train_norm[train_norm.columns[-1]]
x_test_norm, y_test_norm = test_norm[test_norm.columns[:-1]], test_norm[test_norm.columns[-1]]

In [16]:
#chi square method
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(x_train, y_train)
X_train_fs = fs.transform(x_train)
X_test_fs = fs.transform(x_test)

In [17]:
#Matrix değişmedi bu yöntem etkisiz. O yüzden bu yöntem denenmedi.
X_train_fs.shape

(210, 57)

In [18]:
#Mutual Information Features
from sklearn.feature_selection import mutual_info_classif
def mutual_feature_selection(x_train, y_train, x_test, k_var):
  fs = SelectKBest(score_func=mutual_info_classif, k=k_var)
  fs.fit(x_train, y_train)
  X_train_fs = fs.transform(x_train)
  X_test_fs = fs.transform(x_test)
  return X_train_fs, X_test_fs

In [19]:
#grid Search parameters
from sklearn.model_selection import GridSearchCV
knn_parameters = {'n_neighbors' : (3,4,5,6,7), 'weights' : ('uniform', 'distance'), 'algorithm' : ('ball_tree', 'kd_tree', 'brute')}
dt_parameters = {'criterion': ('gini', 'entropy','log_loss'), 'splitter' : ('best', 'random')}
compnb_parameters = {'alpha':(0.0, 0.5, 1.0), 'fit_prior':(True, False), 'norm':(True, False)}
sgdc_parameters = {'loss':('hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'), 'penalty':['l2', 'l1', 'elasticnet']}
gb_paramaeters = {
    "loss":["deviance"],
    "learning_rate": [0.2],
    "min_samples_split": np.linspace(0.2, 1, 4),
    "min_samples_leaf": [0.1],
    "max_depth":[8],
    "max_features":["sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }
    

In [20]:
#Cross-Validation için
from sklearn.model_selection import cross_val_score
X = df_ready[df_ready.columns[:-1]]
Y = df_ready[df_ready.columns[-1]]

X_norm = df_ready[df_ready.columns[:-1]]
Y_norm = df_ready[df_ready.columns[-1]]

In [21]:
#Normal sonuçlar.

methods = {'SGDClassifier' : [sgdc_parameters, SGDClassifier()],
           'Complement Bayes' : [compnb_parameters, ComplementNB()],
           'Decision Tree': [dt_parameters , tree.DecisionTreeClassifier()],
           'Knn' : [knn_parameters, KNeighborsClassifier()],
           'Gradient Boosting':[gb_paramaeters, GradientBoostingClassifier()]
           }

#Modelleri daha sonra kullanmak için saklama listesi
models = []

for i in methods:
    gs_clf = GridSearchCV(methods[i][1], methods[i][0])
    gs_clf.fit(x_train, y_train)
    y_predict = gs_clf.predict(x_test)
    models.append(gs_clf)
    cm = confusion_matrix(y_test, y_predict)
    acc = accuracy_score(y_test, y_predict)
    print(f"For method {i}, accuracy : {acc} - best_params : {gs_clf.best_params_}, cross_val : {cross_val_score(gs_clf, X, Y, cv=5).mean()}")

For method SGDClassifier, accuracy : 0.6 - best_params : {'loss': 'perceptron', 'penalty': 'l1'}, cross_val : 0.6470588235294118
For method Complement Bayes, accuracy : 0.6666666666666666 - best_params : {'alpha': 1.0, 'fit_prior': True, 'norm': True}, cross_val : 0.7294117647058823
For method Decision Tree, accuracy : 0.6 - best_params : {'criterion': 'gini', 'splitter': 'best'}, cross_val : 0.6980392156862745
For method Knn, accuracy : 0.6666666666666666 - best_params : {'algorithm': 'kd_tree', 'n_neighbors': 7, 'weights': 'uniform'}, cross_val : 0.7490196078431373
For method Gradient Boosting, accuracy : 0.6444444444444445 - best_params : {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 10, 'subsample': 1.0}, cross_val : 0.7215686274509804


In [22]:
#Normalize
methods = {'SGDClassifier' : [sgdc_parameters, SGDClassifier()],
           'Complement Bayes' : [compnb_parameters, ComplementNB()],
           'Decision Tree': [dt_parameters , tree.DecisionTreeClassifier()],
           'Knn' : [knn_parameters, KNeighborsClassifier()],
           'Gradient Boosting':[gb_paramaeters, GradientBoostingClassifier()]
           }

for i in methods:
    gs_clf = GridSearchCV(methods[i][1], methods[i][0])
    gs_clf.fit(x_train_norm, y_train_norm)
    y_predict = gs_clf.predict(x_test_norm)
    cm = confusion_matrix(y_test_norm, y_predict)
    acc = accuracy_score(y_test_norm, y_predict)
    print(f"For method {i}, accuracy : {acc} - best_params : {gs_clf.best_params_}, cross_val : {cross_val_score(gs_clf, X_norm, Y_norm, cv=5).mean()}")

For method SGDClassifier, accuracy : 0.6851851851851852 - best_params : {'loss': 'hinge', 'penalty': 'elasticnet'}, cross_val : 0.6862745098039216
For method Complement Bayes, accuracy : 0.7037037037037037 - best_params : {'alpha': 1.0, 'fit_prior': True, 'norm': False}, cross_val : 0.7098039215686274
For method Decision Tree, accuracy : 0.7037037037037037 - best_params : {'criterion': 'entropy', 'splitter': 'best'}, cross_val : 0.6549019607843137
For method Knn, accuracy : 0.7407407407407407 - best_params : {'algorithm': 'brute', 'n_neighbors': 7, 'weights': 'uniform'}, cross_val : 0.7215686274509803
For method Gradient Boosting, accuracy : 0.7037037037037037 - best_params : {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 10, 'subsample': 0.85}, cross_val : 0.7098039215686275


In [47]:
#mutual feature selection
methods = {'SGDClassifier' : [sgdc_parameters, SGDClassifier()],
           'Complement Bayes' : [compnb_parameters, ComplementNB()],
           'Decision Tree': [dt_parameters , tree.DecisionTreeClassifier()],
           'Knn' : [knn_parameters, KNeighborsClassifier()],
           'Gradient Boosting':[gb_paramaeters, GradientBoostingClassifier()]
           }

for k_var in range(15,50,5):
  X_train_fs, X_test_fs = mutual_feature_selection(x_train, y_train, x_test, k_var)
  print(f" **** Best {k_var} feature ****")
  for i in methods:
      gs_clf = GridSearchCV(methods[i][1], methods[i][0])
      gs_clf.fit(X_train_fs, y_train)
      y_predict = gs_clf.predict(X_test_fs)
      cm = confusion_matrix(y_test, y_predict)
      acc = accuracy_score(y_test, y_predict)
      print(f"For method {i}, accuracy : {acc} - best_params : {gs_clf.best_params_}, cross_val : {cross_val_score(gs_clf, X, Y, cv=5).mean()}")
     

 **** Best 15 feature ****
For method SGDClassifier, accuracy : 0.6730769230769231 - best_params : {'loss': 'log', 'penalty': 'elasticnet'}, cross_val : 0.6784313725490196
For method Complement Bayes, accuracy : 0.6730769230769231 - best_params : {'alpha': 0.5, 'fit_prior': True, 'norm': True}, cross_val : 0.7098039215686274
For method Decision Tree, accuracy : 0.6538461538461539 - best_params : {'criterion': 'gini', 'splitter': 'best'}, cross_val : 0.6392156862745096
For method Knn, accuracy : 0.6538461538461539 - best_params : {'algorithm': 'brute', 'n_neighbors': 5, 'weights': 'uniform'}, cross_val : 0.7098039215686275
For method Gradient Boosting, accuracy : 0.7307692307692307 - best_params : {'criterion': 'mae', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 10, 'subsample': 1.0}, cross_val : 0.7411764705882352
 **** Best 20 feature ****
For method SGDClassifier, accuracy : 0.7115

In [23]:
#mutual feature selection
methods = {'SGDClassifier' : [sgdc_parameters, SGDClassifier()],
           'Complement Bayes' : [compnb_parameters, ComplementNB()],
           'Decision Tree': [dt_parameters , tree.DecisionTreeClassifier()],
           'Knn' : [knn_parameters, KNeighborsClassifier()],
           'Gradient Boosting':[gb_paramaeters, GradientBoostingClassifier()]
           }

for k_var in range(2,15,1):
  X_train_fs, X_test_fs = mutual_feature_selection(x_train, y_train, x_test, k_var)
  print(f" **** Best {k_var} feature ****")
  for i in methods:
      gs_clf = GridSearchCV(methods[i][1], methods[i][0])
      gs_clf.fit(X_train_fs, y_train)
      y_predict = gs_clf.predict(X_test_fs)
      cm = confusion_matrix(y_test, y_predict)
      acc = accuracy_score(y_test, y_predict)
      print(f"For method {i}, accuracy : {acc} - best_params : {gs_clf.best_params_}, cross_val : {cross_val_score(gs_clf, X, Y, cv=5).mean()}")

 **** Best 2 feature ****
For method SGDClassifier, accuracy : 0.6481481481481481 - best_params : {'loss': 'log', 'penalty': 'elasticnet'}, cross_val : 0.6666666666666667
For method Complement Bayes, accuracy : 0.6111111111111112 - best_params : {'alpha': 0.0, 'fit_prior': True, 'norm': False}, cross_val : 0.7098039215686274
For method Decision Tree, accuracy : 0.6666666666666666 - best_params : {'criterion': 'gini', 'splitter': 'best'}, cross_val : 0.6862745098039216
For method Knn, accuracy : 0.6666666666666666 - best_params : {'algorithm': 'ball_tree', 'n_neighbors': 6, 'weights': 'uniform'}, cross_val : 0.7215686274509803
For method Gradient Boosting, accuracy : 0.6666666666666666 - best_params : {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 10, 'subsample': 0.5}, cross_val : 0.7254901960784312
 **** Best 3 feature ****
For method SGDClassifier, accur

In [22]:
#Yukarıdaki bütün sonuçları incelediğimizde ortalama 0.70-0.75 arasında dolaştığını görüyoruz.
#Modelleri Ensemble Learning ile Oylama mantığında birleştirelim.
#5 modelden aynı 3 oy geçerli tahmin olarak kabul edilecektir.

preds = []
for model in models:
  preds.append(model.predict(x_test))

In [23]:
preds

[array([0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.]),
 array([0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.]),
 array([1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.]),
 array([0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.]),
 array([0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 

In [35]:
#Sonuçlar sadece 1 ve 0 olabildiğinden 3 ve üstü olan indexlerde sonuç 1 değilse sonuç 0 olarak kabul edilecektir.
toplam = preds[0] + preds[1] + preds[2] + preds[3] + preds[4]
toplam

array([1., 4., 1., 0., 5., 0., 5., 3., 1., 2., 5., 2., 5., 0., 4., 5., 5.,
       3., 2., 5., 0., 1., 0., 4., 5., 0., 4., 4., 3., 3., 4., 5., 4., 2.,
       4., 5., 0., 4., 5., 5., 4., 0., 1., 0., 5.])

In [37]:
ensemble_learning_preds = np.where(toplam<3,0,1)

In [42]:
ensemble_learning_preds

array([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1])

In [38]:
cm = confusion_matrix(y_test, ensemble_learning_preds)
acc = accuracy_score(y_test, ensemble_learning_preds)

In [41]:
cm, acc

(array([[10,  9],
        [ 8, 18]]), 0.6222222222222222)

In [ ]:
#Ensemble learningte başarı maalesef düştü. Demokrasi her zaman tutmuyor demek ki :(